In [ ]:
import torch
from trainer import *
from torch.utils.data import DataLoader
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from models.CNN import StrainEnergyCANN_C, StrainEnergyCANN_polinomial3
from sklearn.metrics import r2_score


# Предобработка данных

In [ ]:
path = r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\data\brain_bade\CANNsBRAINdata.xlsx"

In [ ]:
I1_tc = lambda lam: lam ** 2 + 2.0 / lam
I2_tc = lambda lam: 2.0 * lam + 1 / lam ** 2
I1_s = lambda gam: gam ** 2 + 3.0
F_tc = lambda lam: ([lam, 0, 0], [0, lam **(-0.5), 0], [0, 0, lam**(-0.5)])
F_s = lambda gam: ([1., gam, 0], [0, 1., 0], [0, 0, 1.])


In [ ]:
all_data = pd.read_excel(path, sheet_name="Sheet1", header=[1, 2, 3])
brain_CR_TC_data = all_data.filter(like="CR-comten").copy()
brain_CR_S_data  = all_data.filter(like="CR-shr").copy().dropna(axis=1)

brain_CR_TC_data.columns = brain_CR_TC_data.columns.droplevel(level=[0, 2])
brain_CR_S_data.columns  = brain_CR_S_data.columns.droplevel(level=[0, 2])

mechanical_variables = {
    "I1": [I1_tc, I1_s],
    "I2": [I2_tc, I1_s],
    "F":  [F_tc, F_s],
    # "exp_type": [(lambda x: 1), (lambda x: 0)] # 1 - torsion&compression, 0 - shear
    # "torsion_compression": (lambda x: 1)
}

# calculate I1, I2, F from lambda (torsion&compression and shear)
for variable in mechanical_variables.keys():
    func_calc = mechanical_variables.get(variable)
    brain_CR_TC_data[variable] = brain_CR_TC_data["lambda"].apply(func_calc[0])
    brain_CR_S_data[variable]  = brain_CR_S_data["gamma"].apply(func_calc[1])
    # I1 = pd.concat([brain_CR_TC_data[variable], brain_CR_S_data[variable]], ignore_index=True)
brain_CR_S_data["lambda"] = brain_CR_S_data.pop("gamma")
brain_CR_TC_data["exp_type"] = ["Compression" if i < len(brain_CR_TC_data) / 2 else "Tensile"  for i in range(len(brain_CR_TC_data))]
brain_CR_S_data["exp_type"] = ["Shear"  for i in range(len(brain_CR_S_data))]
data = pd.concat([brain_CR_TC_data, brain_CR_S_data], ignore_index=True)


In [ ]:
combined_data = data
combined_data.columns = ['lambda', 'P_experimental', 'I1', 'I2', 'F', 'experiment_type']

In [ ]:
sns.set(style="whitegrid")
g = sns.relplot(
    data=combined_data,
    x='lambda', y='P_experimental', col='experiment_type', kind='line', height=4, aspect=1.2, facet_kws={'sharey': False, 'sharex': False}
)

# Update axis labels
g.set_axis_labels("Stretch", "Stress, kPa")
g.set_titles("{col_name}")
plt.savefig("brain")

plt.show()


In [ ]:
combined_data

In [ ]:
from torch.utils.data import DataLoader, random_split, Dataset, TensorDataset
import copy 

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # self.features = [dataframe[0],dataframe[2], dataframe[3], dataframe[4], dataframe[5]]
        # self.targets  = dataframe[1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = copy.deepcopy([*self.data.iloc[idx]][:6])
        target = features.pop(1)

        return features, target

    def to_tensor(self):
        for column in self.data.columns:
            if column != "experiment_type":
                self.data[column] = self.data[column].apply(
                    lambda x: torch.tensor(x, dtype=torch.float32)).copy()


In [ ]:
# start = 0
# end = 49
# train_dataset = CustomDataset(combined_data[start:end].copy())
# test_dataset = CustomDataset(combined_data.copy())
# f, t = train_dataset[0]
# lam, i1, i2, F, exp_type = f
# print(f)

In [ ]:
experiments = ["Shear", "Tensile"]
d = pd.concat([combined_data[combined_data["experiment_type"] == experiment] for experiment in experiments]).reset_index(drop=True, inplace=False)
d

In [ ]:
def init_loaders(experiments=["Shear", "Tensile", "Comression"]):
    if type(experiments) == str:
        experiments = [experiments]
    print(experiments)
    
    df = pd.concat([combined_data[combined_data["experiment_type"] == experiment] for experiment in experiments]).reset_index(drop=True, inplace=False)
    train_dataset = CustomDataset(df.copy())
    test_dataset = CustomDataset(combined_data.copy())
    
    train_dataset.to_tensor()
    test_dataset.to_tensor()
    
    train_data_loader = DataLoader(
                            train_dataset,
                            shuffle=True,
                            # num_workers=1,
                            pin_memory=False
    )
    test_data_loader = DataLoader(
                            test_dataset,
                            shuffle=False,
                            # num_workers=1,
                            pin_memory=False
    )

    return train_data_loader, test_data_loader 

In [ ]:
train_data_loader, test_data_loader = init_loaders("Compression")
train_data_loader.dataset.data

# Обучение модели

In [ ]:
experiments=[["Shear", "Tensile", "Comression"],["Tensile", "Comression"], ["Tensile", "Shear"], ["Shear", "Comression"], "Shear", "Tensile", "Comression"]
for experiment in experiments:
    print(experiment)

In [ ]:
# Переименуем столбцы для удобства
# data.columns = ['strain', 'measured_force', 'I1', 'I2', 'F', 'experiment_type', 'predicted_force']

# Удалим ненужные столбцы
# data = data[['strain', 'measured_force', 'experiment_type', 'predicted_force']]
def plot_results(data:pd.DataFrame, plot_name=""):
    # Преобразуем столбец с предсказанной силой в числовой формат
    data['P_model_' + plot_name] = data['P_model_' + plot_name].apply(lambda x: float(str(x).strip('[]')))
    
    # Создадим графики для каждого типа эксперимента
    experiment_types = data['experiment_type'].unique()
    def plot_with_r2(data, experiment_types):
        r2_scores = []
        fig, axes = plt.subplots(1, len(experiment_types), figsize=(15, 6), sharey=False)
        
        for ax, experiment in zip(axes, experiment_types):
            subset = data[data['experiment_type'] == experiment]
            r2 = r2_score(subset['P_experimental'], subset['P_model_' + plot_name])
            
            sns.scatterplot(data=subset, x='lambda', y='P_experimental', label='P_experimental', ax=ax)
            sns.lineplot(data=subset, x='lambda', y='P_model_' + plot_name, label='P_model', color='orange', ax=ax)
            ax.set_title(f'Experiment Type: {experiment}\nR² = {r2:.2f}')
            ax.set_xlabel('Strain')
            ax.set_ylabel('Force (kPa)')
            r2_scores.append(r2)
        
        plt.tight_layout()
        plt.savefig(plot_name)
        plt.show()
        
        return r2_scores
    # Вызовем функцию для построения графиков с r2
    plot_with_r2(data, experiment_types)

In [ ]:
experiments=[["Tensile", "Comression"], ["Tensile", "Shear"], ["Shear", "Comression"], "Shear", "Tensile", "Compression"]
# experiments = ["Tensile", "Comression", "Shear"]
# models = [StrainEnergyCANN_C, StrainEnergyCANN_polinomial3]
models = [StrainEnergyCANN_C]
path = r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models"
best_model_path = r"pretrained_models\brain_StrainEnergyCANN_C\20240523_1820_1400.pth"
for model in models:

    # for idx, experiment in enumerate(experiments):
        train_data_loader, test_data_loader = init_loaders()
        name = "brain_" + str(model.__name__)
        print("----------------------------------------------------------------------")
        print(experiment)
        test_train = Trainer(
            plot_valid=False,
            epochs=3500,
            experiment_name=name,
            l2_reg_coeff=0.01,
            learning_rate=0.001,
            checkpoint=best_model_path,
            model=model
            # dtype = torch.float64
        )
        
        trained_model = test_train.train(train_data_loader, None, weighting_data=True)
    
        trained_model.eval()
        vpredictions = []
        vtargets = []
        for data in test_data_loader:
            features, target = data
            vpredictions.append(trained_model(features).detach().numpy())
        print(trained_model.get_potential())
        
        combined_data["P_model_" + name] = vpredictions
        plot_results(combined_data, name)
        combined_data.to_csv(os.path.join(os.path.join(path, str(name)), "data.csv"))
        combined_data.pop("P_model_" + name)
# trained_model = StrainEnergyCANN_C()


# print("R2:", r2_score_own(vtargets, vpredictions))
# plt.figure(figsize=(10, 5))
# plt.plot(vpredictions, label='P_pred', color='red')
# plt.plot(vtargets, label='P_true', color='black')
# plt.xlabel('lambda/gamma')
# plt.ylabel('P')
# plt.title('Predictions vs. Targets')
# plt.legend()

# plt.show()

In [ ]:
best_model_path = r"C:\Users\drani\dd\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models\PDMS_full\20240522_1718_533.pth"
trained_model = StrainEnergyCANN_polinomial3()
trained_model.load_state_dict(torch.load(best_model_path))
trained_model.eval()
vpredictions = []
vtargets = []
for data in test_data_loader:
    features, target = data
    vpredictions.append(trained_model(features).detach().numpy())
print(trained_model.get_potential())
combined_data["P_model"] = vpredictions

plot_results(combined_data)

In [ ]:
best_model_path = r"C:\Users\drani\dd\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models\PDMS_6term\20240522_1851_466.pth"
trained_model = StrainEnergyCANN_C()
trained_model.load_state_dict(torch.load(best_model_path))
trained_model.eval()
vpredictions = []
vtargets = []
for data in test_data_loader:
    features, target = data
    vpredictions.append(trained_model(features).detach().numpy())
print(trained_model.get_potential())
combined_data["P_model"] = vpredictions

plot_results(combined_data)